## To start the bot, simply open up a terminal and enter ***python echo_bot.py*** to run the bot! Test it by sending commands ('/start' and '/help') and arbitrary text messages.

# Для того чтобы ваш бот мог уведомлять пользователей о временных рамках мероприятий, нужно выполнить следующие шаги:

### План

1. **Настройка источника данных:**
   - Определите, каким образом бот будет получать данные с вашей платформы для выступлений.
   - Если ваш сайт предоставляет API, настройте бота для работы с этим API.
   - Если API не предоставляется, используйте веб-скрейпинг для получения данных.

2. **Создание и настройка базы данных:**
   - Установите и настройте базу данных (например, SQLite, PostgreSQL, MySQL).
   - Создайте таблицы для хранения информации о пользователях, мероприятиях и подписках пользователей на мероприятия.

3. **Изменение логики взаимодействия с пользователями:**
   - Дополните бота командами для подписки и отписки от уведомлений о конкретных мероприятиях.
   - Реализуйте механизм регистрации пользователей.

4. **Функция обработки и отправки уведомлений:**
   - Реализуйте функцию, которая будет проверять текущие мероприятия и отправлять уведомления пользователям в нужное время.
   - Используйте планировщик задач (например, `schedule` или `APScheduler`) для регулярной проверки событий и отправки уведомлений.

5. **Интеграция с основным кодом бота:**
   - Объедините все компоненты и протестируйте бота.


In [ ]:
% pip install pyTelegramBotAPI
% pip install git+https://github.com/eternnoir/pyTelegramBotAPI.git
% pip install pytelegrambotapi --upgrade
% pip install telebot
% pip install requests
% pip install json
% pip install sqlite3
% pip install apscheduler.schedulers.background

In [ ]:
import telebot
import requests
import json
import sqlite3
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta

from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Укажите ваш токен Телеграм-бота
TOKEN = "7474544152:AAFYg5_s8Fyx1Y8HSonGiVwWcpao8yb20ks"

bot = telebot.TeleBot(TOKEN)
scheduler = BackgroundScheduler()
scheduler.start()

# Подсоединение к SQL таблице, в которой хранятся данные.
conn = sqlite3.connect('events.db', check_same_thread=False)
cursor = conn.cursor()

# Создаем таблицы, если их нет
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER UNIQUE
                  )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS subscriptions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    event_id INTEGER,
                    FOREIGN KEY (user_id) REFERENCES users(user_id)
                  )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS events (
                    event_id INTEGER PRIMARY KEY,
                    event_name TEXT,
                    start_time TEXT,
                    end_time TEXT
                  )''')

conn.commit()

# Функция для команды /start
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(
        "Привет 😎 Я чат-бот НАФИ, я помогаю организовать приятные и интересные встречи.\n\n"
        "Узнать, что я могу сделать, ты можешь по команде /info."
    )

# Функция для команды /info
def info(update: Update, context: CallbackContext) -> None:
    update.message.reply_text(
        "Я чат-бот созданный для компании НАФИ, чтобы улучшить взаимодействие спикеров с аудиторией. "
        "Мне можно давать такие задания, как \"Открыть сайт НАФИ\", \"Открыть актуальные исследования\", "
        "\"Узнать подробнее о мероприятии\" и т.д."
    )

# Функция для обработки текстовых сообщений
def text_message(update: Update, context: CallbackContext) -> None:
    text = update.message.text.lower()

    if 'открыть сайт нафи' in text:
        update.message.reply_text("Сайт НАФИ: https://www.nafi.ru")
    elif 'открыть актуальные исследования' in text:
        update.message.reply_text("Актуальные исследования: https://www.nafi.ru/researches/")
    elif 'узнать подробнее о мероприятии' in text:
        update.message.reply_text("\nВведите id мероприятия:")
        # Добавляем обработку id мероприятия
        context.user_data['awaiting_event_id'] = True
    elif context.user_data.get('awaiting_event_id'):
        event_id = text
        context.user_data['awaiting_event_id'] = False
        handle_event_id(update, context, event_id)
    else:
        update.message.reply_text("Извините, я не понял вашу команду. Попробуйте команду /info для получения списка доступных команд.")

def handle_event_id(update: Update, context: CallbackContext, event_id: str):
    cursor.execute("SELECT * FROM events WHERE event_id=?", (event_id,))
    event = cursor.fetchone()
    if event:
        event_info = f"Имя мероприятия: {event[1]}\nНачало: {event[2]}\nОкончание: {event[3]}"
        update.message.reply_text(event_info)
        cursor.execute("INSERT INTO subscriptions (user_id, event_id) VALUES (?, ?)", (update.message.from_user.id, event_id))
        conn.commit()
        update.message.reply_text("Вы подписаны на мероприятия!")
    else:
        update.message.reply_text("Мероприятие с таким id не найдено.")

# Обработка команды /start и /help
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    cursor.execute("INSERT OR IGNORE INTO users (user_id) VALUES (?)", (message.from_user.id,))
    conn.commit()
    bot.reply_to(message, "Привет!!! Я бот НАФИ, моя основная цель - уведомлять пользователей по событиям.")

def main() -> None:
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    # Определяем команды
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("info", info))
    
    # Обработка текстовых сообщений
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, text_message))

    # Запускаем бота
    updater.start_polling()

    # Ожидание завершения
    updater.idle()

if __name__ == "__main__":
    main()

### Описание шагов:
1. **Настройка базы данных:** Сначала создаются таблицы `users`, `subscriptions` и `events` для хранения необходимой информации.
2. **Обработка команд `/subscribe` и `/unsubscribe`:** Команды обрабатываются, чтобы пользователи могли подписываться и отписываться от уведомлений.
3. **Проверка и уведомление о событиях:** Используется APScheduler для регулярной проверки текущих событий и отправки соответствующих уведомлений пользователям.
4. **Интеграция с API:** Функция `fetch_events_from_api` получает данные о мероприятиях с платформы и обновляет базу данных.

Эти шаги помогут вам реализовать функциональность оповещений о мероприятиях в вашем Telegram-боте.